In [4]:
import pandas as pd

In [5]:
weather = pd.read_csv("weather.csv", index_col="DATE")

In [6]:
null_pct = weather.apply(pd.isnull).sum()/weather.shape[0]
valid_columns = weather.columns[null_pct < .05]

In [7]:
weather = weather[valid_columns].copy()
weather.columns = weather.columns.str.lower()

In [8]:
weather.index = pd.to_datetime(weather.index)

In [9]:
weather["tomorrow"] = weather["tmax"].shift(-1)
weather.ffill()
weather = weather.iloc[:-1]
weather.index.name = None

In [10]:
weather = weather[["tmax", "tmin", "prcp", "tomorrow"]]
weather.columns = ["tmax", "tmin", "rain", "tmax_tomorrow"]

In [11]:
sel = weather["2022-11-20":]

In [12]:
sel = sel[["tmax", "tmax_tomorrow"]]

In [13]:
sel["prediction"] = sel["tmax"]

In [14]:
sel.columns = ["tmax", "tmax_tomorrow", "prediction"]

In [15]:
sel = sel[["tmax", "prediction", "tmax_tomorrow"]]

In [16]:
sel

,tmax,prediction,tmax_tomorrow
2022-11-20,61.0,61.0,60.0
2022-11-21,60.0,60.0,62.0
2022-11-22,62.0,62.0,67.0
2022-11-23,67.0,67.0,66.0
2022-11-24,66.0,66.0,70.0
2022-11-25,70.0,70.0,62.0
2022-11-26,62.0,62.0,64.0


In [17]:
sel[["tmax", "prediction"]].to_csv("observations.csv")

In [47]:
sel["error"] = (sel["tmax_tomorrow"] - sel["prediction"]).abs()

In [48]:
sel.to_csv("observation_error.csv")

In [ ]:
# Expert system

In [51]:
lsel = weather["2022-11-15":]
lsel = lsel[["tmax", "tmax_tomorrow"]]

lsel["prediction"] = lsel["tmax"].rolling(5).mean() + ((lsel["tmax"] - lsel["tmax"].shift(1)).apply(lambda x: x>=5)).astype(int) * 2

In [52]:
lsel = lsel["2022-11-20":]
lsel = lsel[["tmax", "prediction", "tmax_tomorrow"]]
lsel["error"] = (lsel["tmax_tomorrow"] - lsel["prediction"]).abs()

In [53]:
lsel["error"].mean()

3.057142857142857

In [54]:
lsel.to_csv("expert_error.csv")

In [ ]:
# Full data

In [55]:
weather = weather.ffill()
weather.to_csv("clean_weather.csv")

In [61]:
shift_weather = weather.copy()
for i in [3,5,7,14,30,60,90]:
    shift_weather[f"tmax_{i}"] = shift_weather["tmax"].rolling(i).mean()
    
shift_weather = shift_weather.dropna()

In [63]:
shift_weather.to_csv("dl_weather.csv")